In [ ]:
# LangGraph 및 관련 라이브러리 설치
!pip install langgraph langchain langchain-openai langchain-community

In [ ]:
# 필수 라이브러리 import
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from typing import TypedDict, Sequence
import os

In [ ]:
# OpenAI API 키 설정 (필요한 경우)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# 기본 설정 확인
print("LangGraph 라이브러리가 성공적으로 설치되었습니다!")
print("사용 가능한 주요 모듈:")
print("- StateGraph: 상태 기반 그래프 생성")
print("- ToolNode: 도구 노드 생성") 
print("- ChatOpenAI: OpenAI 채팅 모델")
print("- BaseMessage, HumanMessage, AIMessage: 메시지 타입들")

In [ ]:
# LangGraph 기본 사용 예제

# 상태 정의
class GraphState(TypedDict):
    messages: Sequence[BaseMessage]

# 간단한 노드 함수 정의
def chatbot_node(state: GraphState):
    """간단한 챗봇 노드"""
    # 마지막 메시지 가져오기
    last_message = state["messages"][-1]
    
    # 간단한 응답 생성 (실제로는 LLM을 사용)
    response = f"안녕하세요! 입력하신 메시지는: '{last_message.content}' 입니다."
    
    return {"messages": [AIMessage(content=response)]}

# 그래프 생성
workflow = StateGraph(GraphState)

# 노드 추가
workflow.add_node("chatbot", chatbot_node)

# 시작점 설정
workflow.set_entry_point("chatbot")

# 종료점 설정
workflow.add_edge("chatbot", END)

# 그래프 컴파일
app = workflow.compile()

print("LangGraph 워크플로우가 성공적으로 생성되었습니다!")

In [ ]:
# 그래프 테스트 실행
initial_state = {
    "messages": [HumanMessage(content="안녕하세요! LangGraph 테스트입니다.")]
}

# 그래프 실행
result = app.invoke(initial_state)

print("입력 메시지:", initial_state["messages"][0].content)
print("응답 메시지:", result["messages"][-1].content)